<a href="https://colab.research.google.com/github/User8549/MyRep/blob/main/%D0%91%D0%B5%D0%B7%D0%BE%D0%BF%D0%B0%D1%81%D0%BD%D0%B0%D1%8F_%D1%80%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%BA%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-8mkczzih
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-8mkczzih
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4304 sha256=2c7636aecffec320d7983a5fa5dfbb7036b3875c08d937fdd93ef6d6b296944a
  Stored in directory: /tmp/pip-ephem-wheel-cache-d4p2biib/wheels/f3/08/cc/e2b5b0e1c92df07dbb50a6f024a68ce090f5e7b2316b41756d
Successfully built NVCCPlugin


In [2]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


Пример запуска

In [ ]:
%%cu
#include <iostream>

__global__ void gpu_f()
{
    printf("hello \n");
}

int main()
{
    cudaDeviceProp dev{};
    cudaGetDeviceProperties(&dev, 0);
    std::cout << "Multiprocessors in device = " << dev.multiProcessorCount << std::endl;
    gpu_f<<<dev.multiProcessorCount, 10>>>();
    cudaDeviceSynchronize();
    return 0;
}

Задание 1
написать программу которая с видеокарты пишет сообщение вида "Hello from a thread b block", где a и b номера нити и блока соответственно

In [3]:
%%cu
#include <iostream>

__global__ void gpu_f()
{
    printf("Hello from block: %d, thread: %d\n",blockIdx.x,threadIdx.x);
}

int main()
{
    gpu_f<<<5, 3>>>();
    cudaDeviceSynchronize();
    return 0;
}

Hello from block: 2, thread: 0
Hello from block: 2, thread: 1
Hello from block: 2, thread: 2
Hello from block: 4, thread: 0
Hello from block: 4, thread: 1
Hello from block: 4, thread: 2
Hello from block: 0, thread: 0
Hello from block: 0, thread: 1
Hello from block: 0, thread: 2
Hello from block: 3, thread: 0
Hello from block: 3, thread: 1
Hello from block: 3, thread: 2
Hello from block: 1, thread: 0
Hello from block: 1, thread: 1
Hello from block: 1, thread: 2



заданиие 2: 
умножение матриц

In [5]:
%%cu
#include <iostream>

__global__ void dev(int* mass1, int* mass2, int* mass3, int n, int m)
{
    int x = threadIdx.x + blockIdx.x * blockDim.x;
    if(x < (n * n)){
        int mm = (int)(x / n);
        int nn = (int)(x % n);
        mass3[x] = 0;
        for(auto j = 0; j < m; j++){
            mass3[x] += mass1[j + m * mm] * mass2[j * n + nn];  
        }
        printf ("x = %d, dev = %d \n", x,mass3[x]);
    }
}

void create(int* mass, int n, int m)
{
    for(auto i = 0; i < n; i++){
        for(auto j = 0; j < m; j++)
        {
            mass[i * m + j] = rand() % 21 - 10;
        }
    }
}
void show(int* mass, int n, int m)
{
    for(auto i = 0; i < n; i++)
    {
        for (auto j = 0; j < m; j++)
        {
            std::cout << mass[i * m + j] << " ";
        }
        std::cout << "\n";
    }
    std::cout << "\n";
}

int main()
{
    int *a, *b, *c;

    int n = 2; int x = 4; //строки
    int m = 4; int y = 2; //столбцы
    
    if (m != x) {
      printf("ERROR SIZE");
      return 0;
    }
 
    cudaMallocManaged(&a,n * m * sizeof(int));
    cudaMallocManaged(&b,x * y * sizeof(int));
    cudaMallocManaged(&c,n * y * sizeof(int));
 
    create(a, n, m);
    create(b, x, y);
 
    show(a, n, m);
    show(b, x, y);
 
    dev<<<n, m>>>(a, b, c, n, m);
    cudaDeviceSynchronize();
    
    std::cout << "\n";
    show(c, n, y);
    cudaFree (a);
    cudaFree (b);
    cudaFree (c);
    return 0;
}


-9 -6 -1 9 
-2 0 0 -1 

5 0 
-8 9 
10 -6 
10 -3 

x = 0, dev = 83 
x = 1, dev = -75 
x = 2, dev = -20 
x = 3, dev = 3 

83 -75 
-20 3 


